# Exploración de Parámetros

Hasta ahora hemos definido métricas que cuantifican el rendimiento de un sistema de bicicletas compartidas.
Pasemos a ver cómo esas métricas dependen de los parámetros del sistema, e.g. la tasa de llegada de clientes a las estaciones.

Además, revisaremos una estrategia de desarrollo de programas, llamada *desarrollo incremental*, que puede ayudarte a escribir programas más rápido y dedicar menos tiempo a depurar.


## Funciones que Devuelven Valores

Hemos usado varias funciones que devuelven valores. Por ejemplo, cuando ejecutamos `sqrt`, devuelve un número que puedes asignar a una variable.


In [ ]:
from numpy import sqrt

root_2 = sqrt(2)
root_2

Y cuando ejecutamos `Estado`, devuelve un nuevo objeto `Estado`:

In [ ]:
bikeshare_state = Estado(robledo=20, c4ta=10)
bikeshare_state


No todas las funciones tienen valores de retorno.
Por ejemplo, cuando ejecutas `paso`, actualiza un objeto `Estado`, pero no devuelve ningún valor.

Para escribir funciones que devuelvan valores, podemos usar una sentencia `return`, así:

In [ ]:
def correr_simulacion(p1, p2, num_pasos):
    estado = Estado(robledo=20, c4ta=10,
                  robledo_vacia=0, c4ta_vacia=0)
    
    for i in range(num_pasos):
        paso(estado, p1, p2)
        
    return estado

Llamamos `correr_simulacion`

In [ ]:
estado_final = correr_simulacion(0.3, 0.2, 60)

El resultado es un objeto `Estado` que representa el estado final del sistema, incluidas las métricas que usaremos para evaluar el rendimiento del sistema:


In [ ]:
print(estado_final.c4ta_vacia, 
      estado_final.robledo_vacia)

La simulación que acabamos de ejecutar comienza con `robledo=20` y `c4ta=10`, y utiliza los valores `p1=0.3`, `p2=0.2` y `num_pasos=60`.
Estos cinco valores son *parámetros del modelo*, que son cantidades que determinan el comportamiento del sistema.

Por ejemplo, la versión anterior de `correr_simulacion` recibe `p1`, `p2` y `num_pasos` como parámetros.
Así que podemos llamar a `correr_simulacion` con diferentes parámetros y ver cómo las métricas, como el número de clientes insatisfechos, dependen de los parámetros.
Pero antes de hacer eso, necesitamos una nueva versión de un bucle `for`.


## Bucles y Arreglos

al llamar `correr_simulacion`, usamos el bucle `for`:

```python
    for i in range(num_pasos):
        paso(estado, p1, p2)
```

En este ejemplo, `range` crea una secuencia de números desde `0` hasta `num_steps` (incluyendo `0` pero no `num_steps`). Cada vez que pasa por el bucle, el siguiente número de la secuencia se asigna a la variable de bucle, `i`. Pero `range` solo funciona con enteros; para obtener una secuencia de valores no enteros, podemos usar `linspace`, que es proporcionado por NumPy:


In [ ]:
from numpy import linspace

p1_array = linspace(0, 1, 5)
p1_array

Los argumentos indican dónde debe comenzar y terminar la secuencia, y cuántos elementos debe contener. En este ejemplo, la secuencia contiene `5` números equidistantes, comenzando en `0` y terminando en `1`.

El resultado es un *array* de NumPy, que es un nuevo tipo de objeto que no hemos visto antes. Un array es un contenedor para una secuencia de números. Podemos usar un array en un bucle `for` de esta manera:

In [ ]:
for p1 in p1_array:
    print(p1)

Cuando se ejecuta este bucle, hace lo siguiente:

1. Obtiene el primer valor del array y lo asigna a `p1`.

2. Ejecuta el cuerpo del bucle, que imprime `p1`.

3. Obtiene el siguiente valor del array y lo asigna a `p1`.

4. Ejecuta el cuerpo del bucle, que imprime `p1`.

5. ...

Y así sucesivamente, hasta llegar al final del array. Esto será útil en la siguiente sección.


## Exploración de Parámetros

Si conocemos los valores reales de parámetros como `p1` y `p2`, podemos usarlos para hacer predicciones específicas, como cuántas bicicletas habrá en **Robledo** después de una hora.

Pero la predicción no es el único objetivo; modelos como este también se usan para explicar por qué los sistemas se comportan como lo hacen y para evaluar diseños alternativos. Por ejemplo, si observamos el sistema y notamos que a menudo nos quedamos sin bicicletas en un momento particular, podríamos usar el modelo para entender por qué sucede eso. Y si estamos considerando agregar más bicicletas, o incluso otra estación, podríamos evaluar el efecto de varios escenarios de “qué pasaría si”.

Como ejemplo, supongamos que tenemos suficientes datos para estimar que `p2` es aproximadamente `0.2`, pero no tenemos información sobre `p1`. Podríamos ejecutar simulaciones con un rango de valores para `p1` y ver cómo varían los resultados. A este proceso se le llama *explorar* un parámetro (*sweeping*), en el sentido de que el valor del parámetro “recorre” un rango de valores posibles.

Ahora que conocemos los bucles y los arrays, podemos usarlos así:


In [ ]:
p1_array = linspace(0, 0.6, 6)
p2 = 0.2
num_steps = 60

for p1 in p1_array:
    final_state = correr_simulacion(p1, p2, num_steps)
    print(p1, final_state.robledo_vacia)

Cada vez que pasa por el bucle, ejecutamos una simulación con un valor diferente de `p1` y el mismo valor de `p2`, `0.2`.
Luego imprimimos `p1` y el número de clientes insatisfechos en **Robledo**.

Para guardar y graficar los resultados, podemos usar un objeto `SweepSeries`, que es similar a un `TimeSeries`; la diferencia es que las etiquetas en un `SweepSeries` son valores de parámetros en lugar de valores de tiempo.

Podemos crear un `SweepSeries` vacío de esta manera:

In [ ]:
def SweepSeries(*args, **kwargs):
    """Crear un objeto pd.Series para almacenar resultados de una exploración de parámetros.

    Args:
        *args: Argumentos pasados a pd.Series.
        **kwargs: Argumentos con nombre pasados a pd.Series.

    Returns:
        pd.Series: Serie con nombre de índice 'Parametro' y nombre 'Metrica'.
    """
    if args or kwargs:
        underride(kwargs, dtype=float)
        series = pd.Series(*args, **kwargs)
    else:
        series = pd.Series([], dtype=np.float64)

    series.index.name = "Parametro"
    if "name" not in kwargs:
        series.name = "Metrica"
    return series


In [ ]:
sweep = SweepSeries()

y añadimos valores de la siguiente forma

In [ ]:
p1_array = linspace(0, 0.6, 31)

for p1 in p1_array:
    estado_final = correr_simulacion(p1, p2, num_steps)
    sweep[p1] = estado_final.robledo_vacia

El resultado es un `SweepSeries` que asigna cada valor de `p1` al número resultante de clientes insatisfechos.

Podemos mostrar los resultados de esta manera:

In [ ]:
print(sweep)

Y podemos graficar los elementos del `SweepSeries` así:

In [ ]:
sweep.plot(label='Robledo', color='C1')

decorar(title='Sistema de Bicicletas Compartidas Robledo-C4TA',
        xlabel='Tasa de llegada de clientes en Robledo (p1 en clientes/min)', 
        ylabel='Número de clientes insatisfechos en Robledo')


El argumento con nombre `color='C1'` especifica el color de la línea.
Los `TimeSeries` que hemos graficado hasta ahora usan el color predeterminado, `C0`, que es azul (ver [https://matplotlib.org/stable/tutorials/colors/colors.html](https://matplotlib.org/stable/tutorials/colors/colors.html) para los demás colores definidos por Matplotlib).
Uso un color diferente para `SweepSeries` como recordatorio de que no es un `TimeSeries`.

Cuando la tasa de llegada en **Robledo** es baja, hay suficientes bicicletas y ningún cliente insatisfecho.
A medida que la tasa de llegada aumenta, es más probable que nos quedemos sin bicicletas y el número de clientes insatisfechos aumenta.
La línea es irregular porque la simulación se basa en números aleatorios. A veces tenemos suerte y hay relativamente pocos clientes insatisfechos; otras veces no tenemos suerte y hay más.

## Desarrollo Incremental

Cuando empiezas a escribir programas que tienen más de unas pocas líneas, puedes encontrarte dedicando más tiempo a la depuración.
Cuanto más código escribas antes de comenzar a depurar, más difícil será encontrar el problema.

El *desarrollo incremental* es una forma de programar que busca minimizar el dolor de la depuración.
Los pasos fundamentales son:

1. **Empieza siempre con un programa que funcione.**
   Si tienes un ejemplo de un libro, o un programa que escribiste y que es similar a lo que estás trabajando, comienza con eso.
   De lo contrario, empieza con algo que *sepas* que es correcto, como `x=5`.
   Ejecuta el programa y confirma que hace lo que esperas.

2. **Haz un solo cambio pequeño y comprobable a la vez.**
   Un cambio "comprobable" es aquel que muestra algo o tiene algún otro efecto que puedas verificar.
   Idealmente, deberías saber cuál es la respuesta correcta, o poder verificarla realizando otro cálculo.

3. **Ejecuta el programa y mira si el cambio funcionó.**
   Si funcionó, vuelve al Paso 2.
   Si no, tendrás que depurar, pero si el cambio que hiciste fue pequeño, no debería llevar mucho tiempo encontrar el problema.

Cuando este proceso funciona, tus cambios suelen funcionar a la primera o, si no, el problema es evidente.
En la práctica, hay dos dificultades con el desarrollo incremental:

* A veces tienes que escribir código adicional para generar una salida visible que puedas verificar.
  A este código adicional se le llama *andamiaje* (*scaffolding*), porque lo usas para construir el programa y luego lo eliminas cuando terminas.
  Puede parecer una pérdida de tiempo, pero el tiempo que dedicas al andamiaje casi siempre es tiempo que ahorras en depuración.

* Cuando estás comenzando, puede que no sea obvio cómo elegir los pasos que te lleven de `x=5` al programa que intentas escribir.
  Verás más ejemplos de este proceso a medida que avancemos, y mejorarás con la experiencia.

Si te encuentras escribiendo más de unas pocas líneas de código antes de empezar a probar, y pasas mucho tiempo depurando, prueba el desarrollo incremental.

## Resumen

Este capítulo introduce funciones que devuelven valores, que usamos para escribir una versión de `correr_simulation` que devuelve un objeto `Estado` con el estado final del sistema.

También introduce `linspace`, que usamos para crear un array de NumPy, y `SweepSeries`, que utilizamos para almacenar los resultados de una exploración de parámetros.

Usamos una exploración de parámetros para analizar la relación entre uno de los parámetros, `p1`, y el número de clientes insatisfechos, que es una métrica que cuantifica qué tan bien (o mal) funciona el sistema.

En los ejercicios, tendrás la oportunidad de explorar otros parámetros y calcular otras métricas.

En el próximo capítulo, pasaremos a un nuevo problema: modelar y predecir el crecimiento de la población mundial.
